In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
from tqdm import tqdm
import pandas as pd
from os.path import expanduser
import sys
from typing import List
import numpy as np
import joblib
from pprint import pprint
import imodelsx.util
from os.path import dirname
import pickle as pkl
import json
from copy import deepcopy
from numpy.linalg import norm
from math import ceil
from imodelsx.qaemb.qaemb import QAEmb, get_sample_questions_and_examples
from neuro.treebank.config import STORIES_POPULAR, STORIES_UNPOPULAR, ECOG_DIR

In [2]:
story_fname = 'cars-2'
features_df = pd.read_csv(
    join(ECOG_DIR, 'data', 'transcripts', story_fname, 'features.csv'))
sec_window = 3
ngram_list = []
for i in tqdm(range(0, len(features_df))):
    row = features_df.iloc[i]
    time_end = row['end']
    time_start = time_end - sec_window
    ngram = features_df[(features_df['end'] >= time_start) & (
        features_df['end'] <= time_end)]['text'].values.tolist()
    ngram_list.append(ngram)
features_df['ngram'] = ngram_list

100%|██████████| 11407/11407 [00:03<00:00, 3094.40it/s]


In [3]:
features_df

,Unnamed: 0,text,lemma,pos,deprel,start,end,sentence,head,bin_head,...,min_mean_pixel_difference,max_global_magnitude,max_global_angle,max_vector_magnitude,max_vector_angle,face_num,max_mean_magnitude,max_median_magnitude,speaker,ngram
0,0,This,this,PRON,nsubj,47.150000,47.320000,This is Agent Leland Turbo.,4,1,...,0.263890,0.129278,343.678017,50.869549,358.020782,0.0,0.169803,0.000000,Leland Turbo,[This]
1,1,is,be,AUX,cop,47.321000,47.420000,This is Agent Leland Turbo.,4,1,...,0.219717,0.077011,299.866566,29.721148,358.020782,0.0,0.108349,0.000000,Leland Turbo,"[This, is]"
2,2,Agent,Agent,NOUN,compound,47.421000,47.710000,This is Agent Leland Turbo.,4,1,...,0.199268,0.088891,291.044357,19.945601,358.020782,1.0,0.104377,0.000000,Leland Turbo,"[This, is, Agent]"
3,3,Leland,leland,PROPN,root,47.791000,48.060000,This is Agent Leland Turbo.,0,0,...,0.143418,0.312056,279.164497,22.096857,339.400940,0.0,0.339084,0.000000,Leland Turbo,"[This, is, Agent, Leland]"
4,4,Turbo,Turbo,PROPN,flat,48.061000,48.360000,This is Agent Leland Turbo.,4,0,...,0.146539,0.312056,280.199287,22.096857,348.823425,0.0,0.339084,0.000000,Leland Turbo,"[This, is, Agent, Leland, Turbo]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11402,11402,if,if,SCONJ,mark,5890.303514,5890.349201,Not if I see you first.,4,1,...,9.913738,8.970222,199.754303,242.723587,354.091858,0.0,10.242637,2.286903,Mater,"[rockets, I, see, you, at, the, finish, line, ..."
11403,11403,I,I,PRON,nsubj,5890.345048,5890.573962,Not if I see you first.,4,1,...,10.511491,8.970222,199.754303,242.723587,354.091858,1.0,10.242637,2.286903,Mater,"[rockets, I, see, you, at, the, finish, line, ..."
11404,11404,see,see,VERB,root,5890.651757,5890.929553,Not if I see you first.,0,0,...,10.126165,8.970222,173.800095,178.414597,357.274017,0.0,10.242637,1.871406,Mater,"[I, see, you, at, the, finish, line, buddy, No..."
11405,11405,you,you,PRON,obj,5890.929712,5891.098882,Not if I see you first.,4,0,...,9.800367,6.199737,318.205860,192.675613,357.274017,1.0,9.007739,1.955157,Mater,"[see, you, at, the, finish, line, buddy, Not, ..."


### Look at story

In [107]:
# remove duplicate consecutive values
df = features_df.loc[features_df['sentence'].shift() !=
                     features_df['sentence']]

# set speaker to '' for duplicate consecutive values
duplicate_speaker = df['speaker'].shift() == df['speaker']
df = df[df['sentence'].notna()]
df.loc[duplicate_speaker, 'speaker'] = ''

# numbered sentences
# df.loc[~duplicate_speaker, 'speaker'] = '<' + df['speaker'] + '>:\n'
# df['sentence_idx'] = np.arange(len(df)) + 1
# df['script'] = df['speaker'] + \
# df['sentence_idx'].astype(str) + '. ' + df['sentence']
# story = '\n'.join(df['script'].iloc[:50])

# unnumbered
df.loc[~duplicate_speaker, 'speaker'] = '\n<' + df['speaker'] + '>: '
df['script'] = df['speaker'] + df['sentence']
story = ' '.join(df['script'].iloc[:500])

In [ ]:
print(story)

### Get popular stories

In [3]:
subject_metadata_files = os.listdir(join(ECOG_DIR, 'data', 'subject_metadata'))
jsons = {f: json.load(open(join(ECOG_DIR, 'data', 'subject_metadata', f)))
         ['title'] for f in subject_metadata_files}
df = pd.DataFrame(jsons.values(), index=jsons.keys()).reset_index()
df.rename(columns={'index': 'filename', 0: 'title'}, inplace=True)
df['subject'] = df['filename'].apply(lambda x: x.split('_trial')[
                                     0].split('_')[-1]).astype(int)
# df = df.sort_values(by='subject')
df = df.sort_values(by='title')

common_subjs = {3, 4, 6, 7, 10}
stories_popular = df[df.subject.isin(common_subjs)].title.unique()
stories_unpopular = df[~df.subject.isin(common_subjs)].title.unique()
print(f'{len(stories_popular)=} {len(stories_unpopular)=}')
stories_popular, stories_unpopular

len(stories_popular)=9 len(stories_unpopular)=12


(array(['Cars 2', 'Coraline', 'Lord Of The Rings 1', 'Lord Of The Rings 2',
        'Megamind', 'Shrek The Third', 'Spiderman Far From Home',
        'The Incredibles', 'Toy Story'], dtype=object),
 array(['Antman', 'Aquaman', 'Avengers Infinity War', 'Black Panther',
        'Fantastic Mr. Fox', 'Guardians Of The Galaxy 2',
        'Guardians Of the Galaxy', 'Sesame Street Episode 3990',
        'Spiderman Homecoming', 'The Martian', 'Thor Ragnarok', 'venom'],
       dtype=object))

In [6]:
df['title'].value_counts()

title
Cars 2                        3
Megamind                      3
Fantastic Mr. Fox             2
Antman                        1
Aquaman                       1
Black Panther                 1
Avengers Infinity War         1
Guardians Of The Galaxy 2     1
Coraline                      1
Guardians Of the Galaxy       1
Lord Of The Rings 1           1
Lord Of The Rings 2           1
Sesame Street Episode 3990    1
Shrek The Third               1
Spiderman Far From Home       1
Spiderman Homecoming          1
The Incredibles               1
The Martian                   1
Thor Ragnarok                 1
Toy Story                     1
venom                         1
Name: count, dtype: int64

In [7]:
df

,filename,title,subject
25,sub_9_trial000_metadata.json,Antman,9
11,sub_2_trial006_metadata.json,Aquaman,2
9,sub_2_trial004_metadata.json,Avengers Infinity War,2
10,sub_2_trial005_metadata.json,Black Panther,2
22,sub_7_trial000_metadata.json,Cars 2,7
0,sub_10_trial000_metadata.json,Cars 2,10
12,sub_3_trial000_metadata.json,Cars 2,3
21,sub_6_trial004_metadata.json,Coraline,6
2,sub_1_trial000_metadata.json,Fantastic Mr. Fox,1
18,sub_5_trial000_metadata.json,Fantastic Mr. Fox,5
